# Moter dataset Anomaly detection

### Check dataset proportions

### dataset split 

To detect anomalies, we plan to input data into the model at 1-second intervals with a sampling rate of 100Hz. Therefore, we will segment the data into 70 samples for each 1-second intervals.

In [4]:
import pandas as pd

label_0 = pd.read_csv("data/label_0_data.csv")
label_1 = pd.read_csv("data/label_1b_data.csv")
label_2 = pd.read_csv("data/label_2_data.csv")
label_3 = pd.read_csv("data/label_3_data.csv")

In [5]:
len_0 = len(label_0)
len_1 = len(label_1)
len_2 = len(label_2)
len_3 = len(label_3)

label_X = [label_0, label_1, label_2, label_3]
rem = [len_0, len_1, len_2, len_3]

for i in range(4):
    r = rem[i] % 100
    label_X[i] = label_X[i].iloc[:rem[i] - r]

    print(len(label_X[i]))



359900
359800
359900
359900


In [2]:
import numpy as np

In [9]:
data_split_0 = np.array_split(label_X[0], len(label_X[0]) // 100)
data_split_1 = np.array_split(label_X[1], len(label_X[1]) // 100)
data_split_2 = np.array_split(label_X[2], len(label_X[2]) // 100)
data_split_3 = np.array_split(label_X[3], len(label_X[3]) // 100)


y_label_0 = np.zeros(len(data_split_0))
y_label_1 = np.ones(len(data_split_1))
y_label_2 = np.full(len(data_split_2), 2)
y_label_3 = np.full(len(data_split_3), 3)

print(len(data_split_1), len(data_split_1[0]))


/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


3598 100


### Split train, validataion, test set

train_ratio = 0.7
validation ratio = 0.15
test ratio = 0.15

In [5]:
# 아래 코드 사용할 것.

train_ratio = 0.7
temp_ratio = 0.3
val_ratio = 0.5
test_ratio = temp_ratio / 2

data_splits = [data_split_0, data_split_1, data_split_2, data_split_3]
y_data_splits = [y_label_0, y_label_1, y_label_2, y_label_3]

train_sizes = []
train_data = []
test_data = []

y_train_data = []
y_test_data = []


for i in range(4):
    data_len = len(data_splits[i])
    train_size = int(data_len * train_ratio)
    train_sizes.append(train_size)
    train_data.append(data_splits[i][:train_size])
    test_data.append(data_splits[i][train_size:])

    y_train_data.append(y_data_splits[i][:train_size])
    y_test_data.append(y_data_splits[i][train_size:])


# check
for i in range(4):
    print(len(train_data[i]) + len(test_data[i]))

x_train_concat = np.concatenate(train_data, axis = 0)
x_test_concat = np.concatenate(test_data, axis = 0)
y_train_concat = np.concatenate(y_train_data, axis=0)
y_test_concat = np.concatenate(y_test_data, axis=0)


val_size = int(x_test_concat.shape[0] * val_ratio)
test_size = x_test_concat.shape[0] - val_size

# validation & test set split
x_val = x_test_concat[:val_size]
x_test = x_test_concat[val_size:]

y_val = y_test_concat[:val_size]
y_test = y_test_concat[val_size:]

# check
print(f"x_val size: {x_val.shape[0]}")
print(f"x_test size: {x_test.shape[0]}")
print(f"y_val size: {y_val.shape[0]}")
print(f"y_test size: {y_test.shape[0]}")

# check 
print(x_train_concat.shape[0] + x_test.shape[0] + x_val.shape[0])
print(y_train_concat.shape[0] + y_test.shape[0] + y_val.shape[0])

3599
3598
3599
3599
x_val size: 2160
x_test size: 2160
y_val size: 2160
y_test size: 2160
14395
14395


In [16]:
train_ratio = 0.7
temp_ratio = 0.3  # for both val and test combined
val_ratio = 0.5  # 50% of temp_ratio for validation
test_ratio = temp_ratio / 2  # same as val_ratio

data_splits = [data_split_0, data_split_1, data_split_2, data_split_3]
y_data_splits = [y_label_0, y_label_1, y_label_2, y_label_3]

train_sizes = []
train_data = []
val_data = []
test_data = []

y_train_data = []
y_val_data = []
y_test_data = []

for i in range(4):
    data_len = len(data_splits[i])
    train_size = int(data_len * train_ratio)
    temp_size = data_len - train_size 
    val_size = int(temp_size * val_ratio)
    
    # Split the data
    train_sizes.append(train_size)
    train_data.append(data_splits[i][:train_size])
    val_data.append(data_splits[i][train_size:train_size + val_size])
    test_data.append(data_splits[i][train_size + val_size:])

    # Split the labels
    y_train_data.append(y_data_splits[i][:train_size])
    y_val_data.append(y_data_splits[i][train_size:train_size + val_size])
    y_test_data.append(y_data_splits[i][train_size + val_size:])

# check the lengths of each split
for i in range(4):
    print(f"Data split {i}: train + val + test =", len(train_data[i]), "+", len(val_data[i]), "+", len(test_data[i]))

# Concatenate all splits across the 4 data parts
x_train_concat = np.concatenate(train_data, axis=0)
x_val_concat = np.concatenate(val_data, axis=0)
x_test_concat = np.concatenate(test_data, axis=0)

y_train_concat = np.concatenate(y_train_data, axis=0)
y_val_concat = np.concatenate(y_val_data, axis=0)
y_test_concat = np.concatenate(y_test_data, axis=0)

# Output final shapes to confirm correct splitting
print("Train data shape:", x_train_concat.shape)
print("Validation data shape:", x_val_concat.shape)
print("Test data shape:", x_test_concat.shape)


Data split 0: train + val + test = 2519 + 540 + 540
Data split 1: train + val + test = 2518 + 540 + 540
Data split 2: train + val + test = 2519 + 540 + 540
Data split 3: train + val + test = 2519 + 540 + 540
Train data shape: (10075, 100, 3)
Validation data shape: (2160, 100, 3)
Test data shape: (2160, 100, 3)


In [17]:
# Count the occurrences of each label in y_val_concat
unique_labels, label_counts = np.unique(y_val_concat, return_counts=True)

# Calculate the percentage of each label
label_ratios = label_counts / len(y_val_concat)

# Print the results
for label, count, ratio in zip(unique_labels, label_counts, label_ratios):
    print(f"Label {label}: count = {count}, ratio = {ratio:.2%}")


Label 0.0: count = 540, ratio = 25.00%
Label 1.0: count = 540, ratio = 25.00%
Label 2.0: count = 540, ratio = 25.00%
Label 3.0: count = 540, ratio = 25.00%


In [18]:
x_train_concat.shape

(10075, 100, 3)

In [19]:
y_train_concat.shape

(10075,)

In [20]:
from keras.utils import to_categorical

# Convert y_train and y_valid to one-hot encoding
y_train_one_hot = to_categorical(y_train_concat, num_classes=4)  # Assuming classes are 0, 1, 2, 3
y_valid_one_hot = to_categorical(y_val_concat, num_classes=4)  # Assuming you have a similar y_valid
y_test_one_hot = to_categorical(y_test_concat, num_classes=4)  # Assuming you have a similar y_test

# Check the shape
print(y_train_one_hot.shape)  # Should be (10070, 4)
print(y_valid_one_hot.shape) 
print(y_test_one_hot.shape) 

(10075, 4)
(2160, 4)
(2160, 4)


In [21]:
# Sum along axis 0 to get the count of each class
class_counts = np.sum(y_train_one_hot, axis=0)

# Calculate the percentage of each class
class_ratios = class_counts / np.sum(class_counts)

# Print the results
for i, (count, ratio) in enumerate(zip(class_counts, class_ratios)):
    print(f"Class {i}: count = {int(count)}, ratio = {ratio:.2%}")


Class 0: count = 2519, ratio = 25.00%
Class 1: count = 2518, ratio = 24.99%
Class 2: count = 2519, ratio = 25.00%
Class 3: count = 2519, ratio = 25.00%


### Define the Model Architecture

In [20]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15294558232718816693
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11378904359985974365
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6933955910558672802
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15573965249812017374
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13699355476082648183
physical_device_desc: "device: XLA_GPU device"
]


In [8]:

# from tensorflow.python.client import device_lib
# print(tf.__version__)
# print(device_lib.list_local_devices())


In [49]:
import tensorflow as tf
import keras
from keras.layers import LSTM, Dropout, Dense, InputLayer
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [50]:
model = keras.models.Sequential([
    # Conv Block 1 
    keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu",
                        input_shape=[100, 3]),
    keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu"), 

    # Pooling layer
    keras.layers.MaxPooling1D(pool_size=2),

    # Conv Block 2 
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu"),
    keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu"),           
    
    # Pooling layer
    keras.layers.MaxPooling1D(pool_size=2),  

    # Conv Block 3
    keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),
    keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"),   
    
    # Pooling layer        
    keras.layers.MaxPooling1D(pool_size=2),  

    # Conv Block 4
    keras.layers.Conv1D(filters=256, kernel_size=3, strides=1, padding="same", activation="relu"),

    keras.layers.GlobalAveragePooling1D(),

    # Updated output layer for multi-class classification
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(4, activation="softmax")  # Change to 4 units and softmax activation
])

# Compile the model
model.compile(loss="categorical_crossentropy",  # Use categorical crossentropy
              optimizer="adam",
              metrics=["accuracy"])  # Accuracy metric is still appropriate

# Train the model
history = model.fit(
    x_train_concat, y_train_one_hot,  # Ensure y_train is one-hot encoded for multi-class
    epochs=20,  
    batch_size=32,
    validation_data=(x_val_concat, y_valid_one_hot),
    verbose=1,
    # class_weight=class_weights_dict,
    # callbacks=[early_stop]
)

# Get predictions
y_pred = model.predict(x_val_concat)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the class with the highest probability

# # Save the model
model.save("conv_net_v1.h5")


Epoch 1/20


/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


315/315 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.4735 - loss: 1.0265 - val_accuracy: 0.9991 - val_loss: 0.0031
Epoch 2/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 1.8578e-04 - val_accuracy: 1.0000 - val_loss: 1.6957e-04
Epoch 3/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 4.7713e-05 - val_accuracy: 0.9995 - val_loss: 3.8967e-04
Epoch 4/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 2.0000e-05 - val_accuracy: 0.9995 - val_loss: 4.6753e-04
Epoch 5/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 1.0970e-05 - val_accuracy: 0.9995 - val_loss: 4.2347e-04
Epoch 6/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 7.3498e-06 - val_accuracy: 0.9995 - val_loss: 3.8241e-04
Epoch 7/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 1.0000 - loss: 5.0721e-06 - val_accuracy: 0.9995 - val_loss: 4.2253e-04
Epoch 8/20
315/315 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 1.0

In [24]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 100, 3)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_161 (Conv1D) │ (None, 100, 32)   │        320 │ input_layer_12[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_162 (Conv1D) │ (None, 100, 32)   │      3,104 │ conv1d_161[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_45    │ (None, 50, 32)    │          0 │ conv1d_162[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_163 (Conv1D) │ (None, 50, 64)    │      6,208 │ max_pooling1d_45… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_164 (Conv1D) │ (None, 50, 64)    │     12,352 │ conv1d_163[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_46    │ (None, 25, 64)    │          0 │ conv1d_164[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_165 (Conv1D) │ (None, 25, 128)   │     24,704 │ max_pooling1d_46… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_166 (Conv1D) │ (None, 25, 128)   │     49,280 │ conv1d_165[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_47    │ (None, 12, 128)   │          0 │ conv1d_166[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_167 (Conv1D) │ (None, 12, 256)   │     98,560 │ max_pooling1d_47… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_168 (Conv1D) │ (None, 12, 256)   │    196,864 │ conv1d_167[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_48    │ (None, 6, 256)    │          0 │ conv1d_168[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_169 (Conv1D) │ (None, 6, 512)    │    393,728 │ max_pooling1d_48… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_170 (Conv1D) │ (None, 6, 512)    │    786,944 │ conv1d_169[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_transpose_27 │ (None, 12, 256)   │    262,400 │ conv1d_170[0][0]  │
│ (Conv1DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_30      │ (None, 12, 512)   │          0 │ conv1d_transpose… │
│ (Concatenate)       │                   │            │ conv1d_168[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_171 (Conv1D) │ (None, 12, 256)   │    393,472 │ concatenate_30[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_172 (Conv1D) │ (None, 12, 256)   │    196,864 │ conv1d_171[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_transpose_28 │ (None, 24, 128)   │     65,664 │ conv1d_172[0][0]  │
│ (Conv1DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cropping1d_14       │ (None, 24, 128)   │          0 │ conv1d_166[0][0]

 Total params: 2,705,156 (10.32 MB)

 Trainable params: 2,705,156 (10.32 MB)

 Non-trainable params: 0 (0.00 B)

In [51]:
loss, accuracy = model.evaluate(x_test_concat, y_test_one_hot, verbose=1)
print(loss)
print(accuracy)


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 2.9157e-07
4.4216082528691913e-07
1.0


In [52]:
y_pred = model.predict(x_test_concat, verbose=1)

print(y_pred[:10])
print(y_test_concat[:10])

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[1.00000000e+00 9.60009633e-28 3.77273608e-08 1.64999848e-15]
 [1.00000000e+00 1.34792505e-27 5.39531690e-08 2.26325255e-15]
 [1.00000000e+00 3.60395750e-29 2.06518336e-09 1.08769433e-16]
 [1.00000000e+00 6.05255383e-29 2.87537216e-09 1.57174428e-16]
 [1.00000000e+00 5.61146893e-28 2.13649454e-08 1.01290370e-15]
 [1.00000000e+00 3.64399427e-28 1.68200565e-08 7.59562237e-16]
 [1.00000000e+00 3.92717723e-28 1.61809055e-08 7.67912023e-16]
 [1.00000000e+00 1.03308351e-28 4.70101469e-09 2.46397226e-16]
 [9.99999881e-01 1.57850104e-27 7.52927320e-08 2.81183175e-15]
 [1.00000000e+00 2.68370634e-28 1.22352048e-08 5.72222372e-16]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [26]:
import numpy as np

# y_pred의 형태 확인 및 정수형으로 변환
if y_pred.ndim == 2:
    y_pred = np.argmax(y_pred, axis=1)
elif y_pred.ndim == 1:
    print("y_pred는 이미 클래스 인덱스입니다.")

# y_test 및 y_pred의 고유 클래스 추출
unique_classes = np.unique(np.concatenate((y_test, y_pred)))  # 두 배열의 고유 클래스 합치기

# 클래스 인덱스를 0부터 시작하도록 조정
class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}
y_test_mapped = np.array([class_to_index[cls] for cls in y_test])
y_pred_mapped = np.array([class_to_index[cls] for cls in y_pred])


print(len(y_test_mapped))
print(len(y_pred_mapped))
# 혼동 행렬 초기화
num_classes = len(unique_classes)
conf_matrix = np.zeros((num_classes, num_classes), dtype=int)

# 혼동 행렬 계산
for i in range(len(y_test)):
    conf_matrix[y_test_mapped[i], y_pred_mapped[i]] += 1

# 혼동 행렬 출력
print("혼동 행렬:\n", conf_matrix)


2160
2160
혼동 행렬:
 [[  0   0   0   0]
 [  0   0   0   0]
 [540 540   0   0]
 [  0   0 540 540]]


In [34]:
len(y_test_concat)

2160

In [37]:
## v1 손대지 말것
from sklearn.metrics import confusion_matrix, classification_report
y_pred_argmax = np.argmax(y_pred, axis=1)

print(classification_report(y_test_concat, y_pred_argmax))

# 혼동 행렬 계산
cm = confusion_matrix(y_test_concat, y_pred_argmax)

# 혼동 행렬 출력
print("혼동 행렬:")
print(cm)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       540
         1.0       1.00      1.00      1.00       540
         2.0       1.00      1.00      1.00       540
         3.0       1.00      1.00      1.00       540

    accuracy                           1.00      2160
   macro avg       1.00      1.00      1.00      2160
weighted avg       1.00      1.00      1.00      2160

혼동 행렬:
[[540   0   0   0]
 [  0 540   0   0]
 [  0   0 540   0]
 [  0   0   0 540]]


In [53]:

## v2
from sklearn.metrics import confusion_matrix, classification_report
y_pred_argmax = np.argmax(y_pred, axis=1)

print(classification_report(y_test_concat, y_pred_argmax))

# 혼동 행렬 계산
cm = confusion_matrix(y_test_concat, y_pred_argmax)

# 혼동 행렬 출력
print("혼동 행렬:")
print(cm)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       540
         1.0       1.00      1.00      1.00       540
         2.0       1.00      1.00      1.00       540
         3.0       1.00      1.00      1.00       540

    accuracy                           1.00      2160
   macro avg       1.00      1.00      1.00      2160
weighted avg       1.00      1.00      1.00      2160

혼동 행렬:
[[540   0   0   0]
 [  0 540   0   0]
 [  0   0 540   0]
 [  0   0   0 540]]


In [54]:
import tensorflow as tf
import numpy as np

# .h5 모델 로드
h5_model = tf.keras.models.load_model('conv_net_v1.h5')

# .tflite 변환기 생성
converter = tf.lite.TFLiteConverter.from_keras_model(h5_model)

# 양자화 적용 (정수 양자화)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter._experimental_disable_per_channel_quantization_for_dense_layers = True

# INT8 양자화를 위해 필요한 대표적인 데이터셋 함수 정의
def representative_dataset():
    for i in range(100):  # 임의로 100개의 샘플을 사용
        # X_windows_balanced에서 일부 샘플을 선택
        sample = x_train_concat[i]  # 각 sample의 shape은 (10, 3)
        # shape을 (1, 10, 3)로 맞추기 위해 배치 차원 추가
        yield [np.expand_dims(sample, axis=0).astype(np.float32)]

# # INT8 양자화를 위해 필요한 대표적인 데이터셋 함수 정의
# def representative_dataset():
#     for _ in range(100):
#         # (1, 10, 24)은 배치 크기 1, 10x24 크기의 입력 데이터를 의미합니다.
#         yield [np.random.rand(1, 10, 24).astype(np.float32)]

# # 대표 데이터셋 지정
converter.representative_dataset = representative_dataset

# 완전한 INT8 양자화를 위한 설정
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # INT8 연산만 허용
converter.inference_input_type = tf.int8  # 입력 타입을 INT8로 설정
converter.inference_output_type = tf.int8  # 출력 타입을 INT8로 설정


# .tflite 모델로 변환
tflite_model = converter.convert()

# .tflite 모델 저장
with open('conv_net_int8_v1.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpnui8pisb/assets


INFO:tensorflow:Assets written to: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpnui8pisb/assets


Saved artifact at '/var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpnui8pisb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  4827672496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827676192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827848320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827850432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827726944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827729936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827732576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827734864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827739088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827741376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4827741552: TensorSpec(s

/Users/seyeong/anaconda3/envs/anomaly_detection/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1727610586.772481 7406712 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727610586.773577 7406712 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-29 20:49:46.777473: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpnui8pisb
2024-09-29 20:49:46.778105: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-29 20:49:46.778110: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/lx/t6fh69qs69d9s21bvsl3_ynr0000gn/T/tmpnui8pisb
2024-09-29 20:49:46.788049: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is no

In [23]:
import numpy as np
from tensorflow import keras

def unet_model(input_shape):
    inputs = keras.Input(shape=input_shape)

    # Encoder (Contracting Path)
    c1 = keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(inputs)
    c1 = keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(c1)
    p1 = keras.layers.MaxPooling1D(pool_size=2)(c1)

    c2 = keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(p1)
    c2 = keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(c2)
    p2 = keras.layers.MaxPooling1D(pool_size=2)(c2)

    c3 = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(p2)
    c3 = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(c3)
    p3 = keras.layers.MaxPooling1D(pool_size=2)(c3)

    c4 = keras.layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(p3)
    c4 = keras.layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(c4)
    p4 = keras.layers.MaxPooling1D(pool_size=2)(c4)

    # Bottleneck
    bottleneck = keras.layers.Conv1D(512, kernel_size=3, padding="same", activation="relu")(p4)
    bottleneck = keras.layers.Conv1D(512, kernel_size=3, padding="same", activation="relu")(bottleneck)

    # Decoder (Expanding Path)
    u4 = keras.layers.Conv1DTranspose(256, kernel_size=2, strides=2, padding="same")(bottleneck)
    u4 = keras.layers.concatenate([u4, c4])
    c5 = keras.layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(u4)
    c5 = keras.layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(c5)

    u3 = keras.layers.Conv1DTranspose(128, kernel_size=2, strides=2, padding="same")(c5)
    # c3_adjusted = keras.layers.Conv1D(128, kernel_size=1, padding="same")(c3) 
    c3_adjusted = keras.layers.Cropping1D(cropping=(1, 0))(c3)  # 마지막 1개 타임스텝 제거

    print(c3_adjusted.shape, u3.shape)
    u3 = keras.layers.concatenate([u3, c3_adjusted])
    c6 = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(u3)
    c6 = keras.layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(c6)

    u2 = keras.layers.Conv1DTranspose(64, kernel_size=2, strides=2, padding="same")(c6)
    c2_adjusted = keras.layers.Cropping1D(cropping=(2, 0))(c2)  # 마지막 1개 타임스텝 제거
    u2 = keras.layers.concatenate([u2, c2_adjusted])
    c7 = keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(u2)
    c7 = keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(c7)

    u1 = keras.layers.Conv1DTranspose(32, kernel_size=2, strides=2, padding="same")(c7)
    c1_adjusted = keras.layers.Cropping1D(cropping=(4, 0))(c1)  # 마지막 1개 타임스텝 제거
    u1 = keras.layers.concatenate([u1, c1_adjusted])
    c8 = keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(u1)
    c8 = keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(c8)

    outputs = keras.layers.Conv1D(4, kernel_size=1, activation="softmax")(c8)  # Change to 4 for multi-class output
    print(outputs.shape)

    model = keras.Model(inputs=inputs, outputs=outputs)

    return model

# Instantiate the model
input_shape = (100, 3)  # Modify based on your actual input shape
model = unet_model(input_shape)

# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

# Train the model
history = model.fit(
    x_train_concat, y_train_one_hot,  # Ensure y_train is one-hot encoded for multi-class
    epochs=20,
    batch_size=32,
    validation_data=(x_val_concat, y_valid_one_hot),
    verbose=1,
    # class_weight=class_weights_dict,
    # callbacks=[early_stop]
)

# Get predictions
y_pred = model.predict(x_val_concat)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the class with the highest probability

# Save the model
model.save("u_net_v1.h5")


(None, 24, 128) (None, 24, 128)
(None, 96, 4)
Epoch 1/20


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 4), output.shape=(None, 96, 4)